<a href="https://colab.research.google.com/github/bforoura/GENAI26/blob/main/Module3/Chat_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **The Chat Blueprint and the Fake Brain**

---
* Here, we set up "Chat" Blueprint.

* Instead of just sending a single sentence to the AI, we create a **ChatPromptTemplate** which organizes information into three distinct layers:

  1. **The System Layer**: It defines the AI's "**personna Instructions**"; e.g., "**You are a professional assistant**"

  2. **The History Layer**: It leaves an **empty placeholder** in the middle of the conversation.

  3. **The Human Layer**: It puts the current request at the very end.
---

* The **Fake Brain or FakeListLLM** is essentially a **proxy** for a real AI like GPT-4 or Gemini.

* When building a complex chain, we often want to make sure the **plumbing** of the code works, i.e., the instructions are formatted correctly and the history is being passed properly without spending money or waiting for a real AI to generate a response.

* Here is how the **Fake Brain** functions:

  * Instead of using billions of parameters to think of an original answer, the Fake Brain takes a list of strings that you provide when you create it.

  * The first time the chain runs, it gives you the first string in your list.

  * The second time it runs, it gives you the second string.

* Even though **it isn't thinking**, it still acts like a **Runnable**.
* This means it can sit perfectly in the middle of your assembly line:

       `Template | Fake Brain | Parser `

* The Template doesn't know the brain is fake; it just sends its list of messages to it.

* The Fake Brain ignores the content of those messages and just hands back the next pre-set answer in its folder.



In [3]:
!pip install -q langchain langchain-community

In [6]:

from langchain_community.llms.fake import FakeListLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# Create a "Fake" Brain
# It will return these answers in order each time you run the chain
responses = ["The job description looks great!", "I remember you mentioned a restaurant."]
llm = FakeListLLM(responses=responses)



# Create the Modern Chat Chain
# Use 'System', 'History', and 'Human'
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a professional assistant."),
    ("placeholder", "{history}"),
    ("human", "Analyze: {job_description}")
])



# The assembly line
chain = chat_prompt_template | llm | StrOutputParser()



# Run the experiment
my_history = [("human", "Hi!"), ("ai", "Hello!")]
result = chain.invoke({"job_description": "Developer", "history": my_history})

print(f"Fake AI Response: {result}")



# Verify the logic
# This shows how the template 'unpacked' the history into a list of 4 messages
prompt_data = chat_prompt_template.invoke({"job_description": "Developer", "history": my_history})
print(f"Total messages in the chain: {len(prompt_data.messages)}")



Fake AI Response: The job description looks great!
Total messages in the chain: 4


# **Message Expansion**

* The above output proves that the **Placeholder** successfully unpacked the history.

  1. **Message 1 (System)**: "You are a professional assistant." This was defined in the template.

  2. **Message 2 (Human History)**: "Hi!". This was pulled from my_history list

  3. **Message 3 (AI History)**: "Hello!" . This was pulled from my_history list

  4. **Message 4 (Current Human)**: "Analyze: Developer". This was created by combining the template with the input